# Data Loading and Preprocessing

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the paths to your dataset directories
train_dir = "/kaggle/input/deepfake-and-real-images/Dataset/Train"
validation_dir = "/kaggle/input/deepfake-and-real-images/Dataset/Validation"
test_dir = "/kaggle/input/deepfake-and-real-images/Dataset"

# Define image data generators for training, validation, and test sets
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Set batch size
batch_size = 64

# Load and preprocess the training dataset
train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(260, 260),  # Xception requires input size of 299x299 , efficientnet- 260x260
    batch_size=batch_size,
    class_mode='binary'  # Set class_mode according to your dataset
)

# Load and preprocess the validation dataset
validation_data = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(260, 260),
    batch_size=batch_size,
    class_mode='binary'
)

# Load and preprocess the test dataset
test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=(260, 260),
    batch_size=batch_size,
    class_mode='binary'
)

2024-05-04 03:20:32.059639: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-04 03:20:32.059721: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-04 03:20:32.189213: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Xception

In [ ]:
import keras
from keras.applications import Xception
from keras.optimizers import Adam
import torch

# Load the pre-trained Xception model without the top layers
base_model = Xception(weights='imagenet', input_shape=(299, 299, 3), include_top=False)

# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

# Add a new output layer for binary classification
x = base_model.output
x = keras.layers.GlobalAveragePooling2D()(x)
outputs = keras.layers.Dense(1, activation='sigmoid')(x)  # 2 classes for binary classification

# Create the fine-tuned model
model = keras.models.Model(inputs=base_model.input, outputs=outputs)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
with tf.device('/GPU:0'):
    history = model.fit(train_data, epochs=10, validation_data=validation_data)

# Evaluate the model on the test set
with tf.device('/GPU:0'):
    loss, accuracy = model.evaluate(test_data)
    print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

# EfficientNetV2M

In [ ]:
import tensorflow.keras as keras
from tensorflow.keras.applications import EfficientNetV2M
from tensorflow.keras.optimizers import Adam

# Step 1: Load the EfficientNetV2-M model
model = EfficientNetV2M(weights='imagenet', input_shape=(260, 260, 3), include_top=False)

# Step 2: Freeze the convolutional base
model.trainable = False

# Step 3: Add custom classification head
x = keras.layers.GlobalAveragePooling2D()(model.output)
outputs = keras.layers.Dense(1, activation='sigmoid')(x)

# Step 4: Compile the model
model = keras.models.Model(inputs=model.input, outputs=outputs)
model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Step 5: Train the model
history = model.fit(train_data, epochs=10, validation_data=validation_data)

# Step 6: Evaluate the model
loss, accuracy = model.evaluate(test_data)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')
